<a target="_blank" href="https://colab.research.google.com/github/cerr/pyCERR-Notebooks/blob/main/copy_seg_to_scan.ipynb">
  <img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/>
</a>

## Copy segmentation from one scan to another

This notebook demonstrates copying segmentation from one scan to another. Scan and segmentation in the example are read from Nifti, but can also be loaded from DICOM format.

#### Define Paths for scans and segmentation

In [ ]:
origNiiScan = r'path/to/original_scan.nii'
origNiiStr = r'path/to/original_seg.nii'
resampNiiScan = r'path/to/smore_resampled_scan.nii'
outputNiiStr = r'path/to/output_resampled_seg.nii.gz'

#### Import scans and segmentation to planC

In [ ]:
from cerr import plan_container as pc
planC = pc.load_nii_scan(origNiiScan)
planC = pc.load_nii_scan(resampNiiScan, planC)
assocScanNum = 0 # index of the original scan
structName = 'tumor'
planC = pc.load_nii_structure(origNiiStr, assocScanNum, planC, labels_dict={1: structName})

In [ ]:
print("Number of scans:" + str(len(planC.scan)))
print("Number of structures:" + str(len(planC.structure)))

#### Copy segmentation from the original scan to the resampled scan

In [ ]:
from cerr.dataclasses import structure as cerrStruct
structNum = 0
scanNum = 1
planC = cerrStruct.copyToScan(structNum, scanNum, planC)
print("Number of structures:" + str(len(planC.structure)))

#### Split segmentations by connected components

In [ ]:
import cerr.contour.rasterseg as rs
from skimage import measure
newStrNum = len(planC.structure)-1
mask3M = rs.getStrMask(newStrNum, planC)    
labeledMask3M, numLabels = measure.label(mask3M, return_num = True)
for label in range(1,numLabels+1):
    maskForLabel3M = labeledMask3M == label
    strName = 'comp ' + str(label)
    planC = cerrStruct.import_mask(maskForLabel3M, scanNum, strName, planC)

#### Visualize scan and segmentation

In [ ]:
# The following snippet visualizes the last structure in the list. Multiple structures can be passed in strNumList.
from cerr import viewer as vwr
import numpy as np
scanNumList = [1] # list of scans
doseNumList = [] #  list of doses
strNumList = [len(planC.structure)-1] # list of structures
displayMode = '2d' # 'path' or 'surface'
viewer, scan_layer, dose_layer, struct_layer = \
    vwr.show_scan_struct_dose(scanNumList, strNumList, doseNumList, planC, displayMode)

#### Export segmentation to Nifti

In [ ]:
# The following snippet assumes that the structure to export is the last one in the list.
# Update the snipet to iterate over a list of structures to export multiple strucutre files.
structNum = len(planC.structure)-1 
planC.structure[structNum].save_nii(outputNiiStr, planC)